# Downloads

In [0]:
!pip install tensorboardX

# **Imports**

In [0]:
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.python.keras import backend as keras
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import *
from tensorflow.python.keras.optimizers import *
from google.colab import files
import os
from os.path import join as pjoin
import collections
import json
import torch
import numpy as np
import scipy.misc as m
import scipy.io as io
import matplotlib.pyplot as plt
import glob
import tensorflow
from numpy import asarray
import tensorflow as tf

import time
import torch
import torch.nn as nn
import copy
import logging
from torch.optim import SGD, Adam, ASGD, Adamax, Adadelta, Adagrad, RMSprop
import torch.optim
from torch.optim.lr_scheduler import MultiStepLR, ExponentialLR, CosineAnnealingLR
import torch.nn.functional as F
import functools
from tensorboardX import SummaryWriter

from PIL import Image
from tqdm import tqdm

from torch.utils import data
from torchvision import transforms

disable_eager_execution()

# **Get** **the** **Data**

In [0]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
!tar -xf VOCtrainval_11-May-2012.tar


# **Preprocessing**

In [0]:
class pascalVOCLoader(data.Dataset):
    """Data loader for the Pascal VOC semantic segmentation dataset.
    Annotations from both the original VOC data (which consist of RGB images
    in which colours map to specific classes) and the SBD (Berkely) dataset
    (where annotations are stored as .mat files) are converted into a common
    `label_mask` format.  Under this format, each mask is an (M,N) array of
    integer values from 0 to 21, where 0 represents the background class.
    The label masks are stored in a new folder, called `pre_encoded`, which
    is added as a subdirectory of the `SegmentationClass` folder in the
    original Pascal VOC data layout.
    A total of five data splits are provided for working with the VOC data:
        train: The original VOC 2012 training data - 1464 images
        val: The original VOC 2012 validation data - 1449 images
        trainval: The combination of `train` and `val` - 2913 images
        train_aug: The unique images present in both the train split and
                   training images from SBD: - 8829 images (the unique members
                   of the result of combining lists of length 1464 and 8498)
        train_aug_val: The original VOC 2012 validation data minus the images
                   present in `train_aug` (This is done with the same logic as
                   the validation set used in FCN PAMI paper, but with VOC 2012
                   rather than VOC 2011) - 904 images
    """

    def __init__(self,root,sbd_path=None,split="train_aug",is_transform=False,img_size=512,
    augmentations=None,img_norm=True,
        test_mode=False,):
        self.root = root
        self.sbd_path = sbd_path
        self.split = split
        self.is_transform = is_transform
        self.augmentations = augmentations
        self.img_norm = img_norm
        self.test_mode = test_mode
        self.n_classes = 21
        self.mean = np.array([104.00699, 116.66877, 122.67892])
        self.files = collections.defaultdict(list)
        self.img_size = img_size if isinstance(img_size, tuple) else (img_size, img_size)

        if not self.test_mode:
            for split in ["train", "val", "trainval"]:
                path = pjoin(self.root, "ImageSets/Segmentation", split + ".txt")
                file_list = tuple(open(path, "r"))
                file_list = [id_.rstrip() for id_ in file_list]
                self.files[split] = file_list
            #self.setup_annotations()
        self.tf = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),  
            ]
        )

    def __len__(self):
      return len(self.files[self.split])

    def __getitem__(self, index):
      im_name = self.files[self.split][index]
      im_path = pjoin(self.root, "JPEGImages", im_name + ".jpg")
      lbl_path = pjoin(self.root, "SegmentationClass/", im_name + ".png")
    #   lbl_path = pjoin(self.root, "SegmentationClass/pre_encoded", im_name + ".png")
      im = Image.open(im_path)
      lbl = Image.open(lbl_path)
      if self.augmentations is not None:
        im, lbl = self.augmentations(im, lbl)
      if self.is_transform:
        im, lbl = self.transform(im, lbl)
      return im, lbl

    def transform(self, img, lbl):
        if self.img_size == ("same", "same"):
            pass
        else:
            # img = np.array(img) 
            # lbl = np.array(lbl) 
            img = img.resize((self.img_size[0], self.img_size[1]))  # uint8 with RGB mode
            lbl = lbl.resize((self.img_size[0], self.img_size[1]))
            # img = tf.image.convert_image_dtype(img, tf.float64, saturate=False, name=None)
            # lbl = tf.image.convert_image_dtype(lbl, tf.float64, saturate=False, name=None)

        # print("IMG:" + str(type(img)))
        # print("LBL:" + str(type(lbl)))
        img = self.tf(img)
        # img = img.numpy()
        # img = asarray(img)
        # img = tensorflow.keras.preprocessing.image.img_to_array(img, data_format=None, dtype=tensorflow.float64)
        lbl = torch.from_numpy(np.array(lbl)).long()
        lbl[lbl == 255] = 0
        # lbl = asarray(lbl)
        # print("IMG:" + str(type(img)))
        # print("LBL:" + str(type(lbl)))
        return img, lbl

    def get_pascal_labels(self):
        """Load the mapping that associates pascal classes with label colors
        Returns:
            np.ndarray with dimensions (21, 3)
        """
        return np.asarray(
            [
                [0, 0, 0],
                [128, 0, 0],
                [0, 128, 0],
                [128, 128, 0],
                [0, 0, 128],
                [128, 0, 128],
                [0, 128, 128],
                [128, 128, 128],
                [64, 0, 0],
                [192, 0, 0],
                [64, 128, 0],
                [192, 128, 0],
                [64, 0, 128],
                [192, 0, 128],
                [64, 128, 128],
                [192, 128, 128],
                [0, 64, 0],
                [128, 64, 0],
                [0, 192, 0],
                [128, 192, 0],
                [0, 64, 128],
            ]
        )

    def encode_segmap(self, mask):
        """Encode segmentation label images as pascal classes
        Args:
            mask (np.ndarray): raw segmentation label image of dimension
              (M, N, 3), in which the Pascal classes are encoded as colours.
        Returns:
            (np.ndarray): class map with dimensions (M,N), where the value at
            a given location is the integer denoting the class index.
        """
        mask = mask.astype(int)
        label_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.int16)
        for ii, label in enumerate(self.get_pascal_labels()):
            label_mask[np.where(np.all(mask == label, axis=-1))[:2]] = ii
        label_mask = label_mask.astype(int)
        return label_mask

    def decode_segmap(self, label_mask, plot=False):
        """Decode segmentation class labels into a color image
        Args:
            label_mask (np.ndarray): an (M,N) array of integer values denoting
              the class label at each spatial location.
            plot (bool, optional): whether to show the resulting color image
              in a figure.
        Returns:
            (np.ndarray, optional): the resulting decoded color image.
        """
        label_colours = self.get_pascal_labels()
        r = label_mask.copy()
        g = label_mask.copy()
        b = label_mask.copy()
        for ll in range(0, self.n_classes):
            r[label_mask == ll] = label_colours[ll, 0]
            g[label_mask == ll] = label_colours[ll, 1]
            b[label_mask == ll] = label_colours[ll, 2]
        rgb = np.zeros((label_mask.shape[0], label_mask.shape[1], 3))
        rgb[:, :, 0] = r / 255.0
        rgb[:, :, 1] = g / 255.0
        rgb[:, :, 2] = b / 255.0
        if plot:
            plt.imshow(rgb)
            plt.show()
        else:
            return rgb

    def setup_annotations(self):
        """Sets up Berkley annotations by adding image indices to the
        `train_aug` split and pre-encode all segmentation labels into the
        common label_mask format (if this has not already been done). This
        function also defines the `train_aug` and `train_aug_val` data splits
        according to the description in the class docstring
        """
        sbd_path = self.sbd_path
        target_path = pjoin(self.root, "SegmentationClass/pre_encoded")
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        path = pjoin(sbd_path, "dataset/train.txt")
        sbd_train_list = tuple(open(path, "r"))
        sbd_train_list = [id_.rstrip() for id_ in sbd_train_list]
        train_aug = self.files["train"] + sbd_train_list

        # keep unique elements (stable)
        train_aug = [train_aug[i] for i in sorted(np.unique(train_aug, return_index=True)[1])]
        self.files["train_aug"] = train_aug
        set_diff = set(self.files["val"]) - set(train_aug)  # remove overlap
        self.files["train_aug_val"] = list(set_diff)

        pre_encoded = glob.glob(pjoin(target_path, "*.png"))
        expected = np.unique(self.files["train_aug"] + self.files["val"]).size

        if len(pre_encoded) != expected:
            print("Pre-encoding segmentation masks...")
            for ii in tqdm(sbd_train_list):
                lbl_path = pjoin(sbd_path, "dataset/cls", ii + ".mat")
                data = io.loadmat(lbl_path)
                lbl = data["GTcls"][0]["Segmentation"][0].astype(np.int32)
                lbl = m.toimage(lbl, high=lbl.max(), low=lbl.min())
                m.imsave(pjoin(target_path, ii + ".png"), lbl)

            for ii in tqdm(self.files["trainval"]):
                fname = ii + ".png"
                lbl_path = pjoin(self.root, "SegmentationClass", fname)
                lbl = self.encode_segmap(m.imread(lbl_path))
                lbl = m.toimage(lbl, high=lbl.max(), low=lbl.min())
                m.imsave(pjoin(target_path, fname), lbl)

        assert expected == 9733, "unexpected dataset sizes"


In [0]:
class Compose(object):
    def __init__(self, augmentations):
        self.augmentations = augmentations
        self.PIL2Numpy = False

    def __call__(self, img, mask):
        if isinstance(img, np.ndarray):
            img = Image.fromarray(img, mode="RGB")
            mask = Image.fromarray(mask, mode="L")
            self.PIL2Numpy = True

        assert img.size == mask.size
        for a in self.augmentations:
            img, mask = a(img, mask)

        if self.PIL2Numpy:
            img, mask = np.array(img), np.array(mask, dtype=np.uint8)

        return img, mask


class RandomHorizontallyFlip(object):
    def __init__(self, p):
        self.p = p

    def __call__(self, img, mask):
        if random.random() < self.p:
            return (img.transpose(Image.FLIP_LEFT_RIGHT), mask.transpose(Image.FLIP_LEFT_RIGHT))
        return img, mask


class RandomRotate(object):
    def __init__(self, degree):
        self.degree = degree

    def __call__(self, img, mask):
        rotate_degree = random.random() * 2 * self.degree - self.degree
        return (
            tf.affine(
                img,
                translate=(0, 0),
                scale=1.0,
                angle=rotate_degree,
                resample=Image.BILINEAR,
                fillcolor=(0, 0, 0),
                shear=0.0,
            ),
            tf.affine(
                mask,
                translate=(0, 0),
                scale=1.0,
                angle=rotate_degree,
                resample=Image.NEAREST,
                fillcolor=250,
                shear=0.0,
            ),
        )

# **Model**

In [0]:
class fcn8s(nn.Module):
    def __init__(self, n_classes=21, learned_billinear=True):
        super(fcn8s, self).__init__()
        self.learned_billinear = learned_billinear
        self.n_classes = n_classes
        self.loss = functools.partial(cross_entropy2d, size_average=False)

        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=100),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2, ceil_mode=True),
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2, ceil_mode=True),
        )

        self.conv_block3 = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2, ceil_mode=True),
        )

        self.conv_block4 = nn.Sequential(
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2, ceil_mode=True),
        )

        self.conv_block5 = nn.Sequential(
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2, ceil_mode=True),
        )

        self.classifier = nn.Sequential(
            nn.Conv2d(512, 4096, 7),
            nn.ReLU(inplace=True),
            nn.Dropout2d(),
            nn.Conv2d(4096, 4096, 1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(),
            nn.Conv2d(4096, self.n_classes, 1),
        )

        self.score_pool4 = nn.Conv2d(512, self.n_classes, 1)
        self.score_pool3 = nn.Conv2d(256, self.n_classes, 1)

        if self.learned_billinear:
            self.upscore2 = nn.ConvTranspose2d(
                self.n_classes, self.n_classes, 4, stride=2, bias=False
            )
            self.upscore4 = nn.ConvTranspose2d(
                self.n_classes, self.n_classes, 4, stride=2, bias=False
            )
            self.upscore8 = nn.ConvTranspose2d(
                self.n_classes, self.n_classes, 16, stride=8, bias=False
            )

        for m in self.modules():
            if isinstance(m, nn.ConvTranspose2d):
                m.weight.data.copy_(
                    get_upsampling_weight(m.in_channels, m.out_channels, m.kernel_size[0])
                )

    def forward(self, x):
        conv1 = self.conv_block1(x)
        conv2 = self.conv_block2(conv1)
        conv3 = self.conv_block3(conv2)
        conv4 = self.conv_block4(conv3)
        conv5 = self.conv_block5(conv4)

        score = self.classifier(conv5)

        if self.learned_billinear:
            upscore2 = self.upscore2(score)
            score_pool4c = self.score_pool4(conv4)[
                :, :, 5 : 5 + upscore2.size()[2], 5 : 5 + upscore2.size()[3]
            ]
            upscore_pool4 = self.upscore4(upscore2 + score_pool4c)

            score_pool3c = self.score_pool3(conv3)[
                :, :, 9 : 9 + upscore_pool4.size()[2], 9 : 9 + upscore_pool4.size()[3]
            ]

            out = self.upscore8(score_pool3c + upscore_pool4)[
                :, :, 31 : 31 + x.size()[2], 31 : 31 + x.size()[3]
            ]
            return out.contiguous()

        else:
            score_pool4 = self.score_pool4(conv4)
            score_pool3 = self.score_pool3(conv3)
            score = F.upsample(score, score_pool4.size()[2:])
            score += score_pool4
            score = F.upsample(score, score_pool3.size()[2:])
            score += score_pool3
            out = F.upsample(score, x.size()[2:])

        return out

    def init_vgg16_params(self, vgg16, copy_fc8=True):
        blocks = [
            self.conv_block1,
            self.conv_block2,
            self.conv_block3,
            self.conv_block4,
            self.conv_block5,
        ]

        ranges = [[0, 4], [5, 9], [10, 16], [17, 23], [24, 29]]
        features = list(vgg16.features.children())

        for idx, conv_block in enumerate(blocks):
            for l1, l2 in zip(features[ranges[idx][0] : ranges[idx][1]], conv_block):
                if isinstance(l1, nn.Conv2d) and isinstance(l2, nn.Conv2d):
                    assert l1.weight.size() == l2.weight.size()
                    assert l1.bias.size() == l2.bias.size()
                    l2.weight.data = l1.weight.data
                    l2.bias.data = l1.bias.data
        for i1, i2 in zip([0, 3], [0, 3]):
            l1 = vgg16.classifier[i1]
            l2 = self.classifier[i2]
            l2.weight.data = l1.weight.data.view(l2.weight.size())
            l2.bias.data = l1.bias.data.view(l2.bias.size())
        n_class = self.classifier[6].weight.size()[0]
        if copy_fc8:
            l1 = vgg16.classifier[6]
            l2 = self.classifier[6]
            l2.weight.data = l1.weight.data[:n_class, :].view(l2.weight.size())
            l2.bias.data = l1.bias.data[:n_class]


In [0]:
def get_upsampling_weight(in_channels, out_channels, kernel_size):
    """Make a 2D bilinear kernel suitable for upsampling"""
    factor = (kernel_size + 1) // 2
    if kernel_size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = np.ogrid[:kernel_size, :kernel_size]
    filt = (1 - abs(og[0] - center) / factor) * (1 - abs(og[1] - center) / factor)
    weight = np.zeros((in_channels, out_channels, kernel_size, kernel_size), dtype=np.float64)
    weight[range(in_channels), range(out_channels), :, :] = filt
    return torch.from_numpy(weight).float()

## Loss

In [0]:
def cross_entropy2d(input, target, weight=None, size_average=True):
    n, c, h, w = input.size()
    nt, ht, wt = target.size()

    # Handle inconsistent size between input and target
    if h != ht and w != wt:  # upsample labels
        input = F.interpolate(input, size=(ht, wt), mode="bilinear", align_corners=True)

    input = input.transpose(1, 2).transpose(2, 3).contiguous().view(-1, c)
    target = target.view(-1)
    loss = F.cross_entropy(
        input, target, weight=weight, size_average=size_average, ignore_index=250
    )
    return loss


def multi_scale_cross_entropy2d(input, target, weight=None, size_average=True, scale_weight=None):
    if not isinstance(input, tuple):
        return cross_entropy2d(input=input, target=target, weight=weight, size_average=size_average)

    # Auxiliary training for PSPNet [1.0, 0.4] and ICNet [1.0, 0.4, 0.16]
    if scale_weight is None:  # scale_weight: torch tensor type
        n_inp = len(input)
        scale = 0.4
        scale_weight = torch.pow(scale * torch.ones(n_inp), torch.arange(n_inp).float()).to(
            target.device
        )

    loss = 0.0
    for i, inp in enumerate(input):
        loss = loss + scale_weight[i] * cross_entropy2d(
            input=inp, target=target, weight=weight, size_average=size_average
        )

    return loss


def bootstrapped_cross_entropy2d(input, target, K, weight=None, size_average=True):

    batch_size = input.size()[0]

    def _bootstrap_xentropy_single(input, target, K, weight=None, size_average=True):

        n, c, h, w = input.size()
        input = input.transpose(1, 2).transpose(2, 3).contiguous().view(-1, c)
        target = target.view(-1)
        loss = F.cross_entropy(
            input, target, weight=weight, reduce=False, size_average=False, ignore_index=250
        )

        topk_loss, _ = loss.topk(K)
        reduced_topk_loss = topk_loss.sum() / K

        return reduced_topk_loss

    loss = 0.0
    # Bootstrap from each image not entire batch
    for i in range(batch_size):
        loss += _bootstrap_xentropy_single(
            input=torch.unsqueeze(input[i], 0),
            target=torch.unsqueeze(target[i], 0),
            K=K,
            weight=weight,
            size_average=size_average,
        )
    return loss / float(batch_size)

In [0]:
logger = logging.getLogger("ptsemseg")

lossdict = {
    "cross_entropy": cross_entropy2d,
    "bootstrapped_cross_entropy": bootstrapped_cross_entropy2d,
    "multi_scale_cross_entropy": multi_scale_cross_entropy2d,
}


def get_loss_function(loss_name, size_average):
    if loss_name is None:
        logger.info("Using default cross entropy loss")
        return cross_entropy2d

    else:
        loss_params = {"size_average": size_average}

        if loss_name not in lossdict:
            raise NotImplementedError("Loss {} not implemented".format(loss_name))

        logger.info("Using {} with {} params".format(loss_name, loss_params))
        return functools.partial(lossdict[loss_name], **loss_params)

## Optimizer

In [0]:
logger = logging.getLogger("ptsemseg")


def get_optimizer(model, name, lr, weight_decay):
    optdict = {
    "sgd": SGD,
    "adam": Adam,
    "asgd": ASGD,
    "adamax": Adamax,
    "adadelta": Adadelta,
    "adagrad": Adagrad,
    "rmsprop": RMSprop,
    }

    if name not in optdict:
        raise NotImplementedError("Optimizer {} not implemented".format(name))

    logger.info("Using {} optimizer".format(name))

    return optdict[name](model.parameters(), lr=lr, weight_decay=weight_decay)


## Scheduler

In [0]:
class ConstantLR(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, last_epoch=-1):
        super(ConstantLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        return [base_lr for base_lr in self.base_lrs]


class PolynomialLR(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, max_iter, decay_iter=1, gamma=0.9, last_epoch=-1):
        self.decay_iter = decay_iter
        self.max_iter = max_iter
        self.gamma = gamma
        super(PolynomialLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        if self.last_epoch % self.decay_iter or self.last_epoch % self.max_iter:
            return [base_lr for base_lr in self.base_lrs]
        else:
            factor = (1 - self.last_epoch / float(self.max_iter)) ** self.gamma
            return [base_lr * factor for base_lr in self.base_lrs]


class WarmUpLR(torch.optim.lr_scheduler._LRScheduler):
    def __init__(
        self, optimizer, scheduler, mode="linear", warmup_iters=100, gamma=0.2, last_epoch=-1
    ):
        self.mode = mode
        self.scheduler = scheduler
        self.warmup_iters = warmup_iters
        self.gamma = gamma
        super(WarmUpLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        cold_lrs = self.scheduler.get_lr()

        if self.last_epoch < self.warmup_iters:
            if self.mode == "linear":
                alpha = self.last_epoch / float(self.warmup_iters)
                factor = self.gamma * (1 - alpha) + alpha

            elif self.mode == "constant":
                factor = self.gamma
            else:
                raise KeyError("WarmUp type {} not implemented".format(self.mode))

            return [factor * base_lr for base_lr in cold_lrs]

        return cold_lrs
        

In [0]:
schedulerdict = {
    "constant_lr": ConstantLR,
    "poly_lr": PolynomialLR,
    "multi_step": MultiStepLR,
    "cosine_annealing": CosineAnnealingLR,
    "exp_lr": ExponentialLR,
}


def get_scheduler(optimizer, scheduler_dict):
    if scheduler_dict is None:
        logger.info("Using No LR Scheduling")
        return ConstantLR(optimizer)

    s_type = scheduler_dict["name"]
    scheduler_dict.pop("name")

    logging.info("Using {} scheduler with {} params".format(s_type, scheduler_dict))

    warmup_dict = {}
    if "warmup_iters" in scheduler_dict:
        # This can be done in a more pythonic way...
        warmup_dict["warmup_iters"] = scheduler_dict.get("warmup_iters", 100)
        warmup_dict["mode"] = scheduler_dict.get("warmup_mode", "linear")
        warmup_dict["gamma"] = scheduler_dict.get("warmup_factor", 0.2)

        logger.info(
            "Using Warmup with {} iters {} gamma and {} mode".format(
                warmup_dict["warmup_iters"], warmup_dict["gamma"], warmup_dict["mode"]
            )
        )

        scheduler_dict.pop("warmup_iters", None)
        scheduler_dict.pop("warmup_mode", None)
        scheduler_dict.pop("warmup_factor", None)

        base_scheduler = schedulerdict[s_type](optimizer, **scheduler_dict)
        return WarmUpLR(optimizer, base_scheduler, **warmup_dict)

    return schedulerdict[s_type](optimizer, **scheduler_dict)

## Metrics

In [0]:
class runningScore(object):
    def __init__(self, n_classes):
        self.n_classes = n_classes
        self.confusion_matrix = np.zeros((n_classes, n_classes))

    def _fast_hist(self, label_true, label_pred, n_class):
        mask = (label_true >= 0) & (label_true < n_class)
        hist = np.bincount(
            n_class * label_true[mask].astype(int) + label_pred[mask], minlength=n_class ** 2
        ).reshape(n_class, n_class)
        return hist

    def update(self, label_trues, label_preds):
        for lt, lp in zip(label_trues, label_preds):
            self.confusion_matrix += self._fast_hist(lt.flatten(), lp.flatten(), self.n_classes)

    def get_scores(self):
        """Returns accuracy score evaluation result.
            - overall accuracy
            - mean accuracy
            - mean IU
            - fwavacc
        """
        hist = self.confusion_matrix
        acc = np.diag(hist).sum() / hist.sum()
        acc_cls = np.diag(hist) / hist.sum(axis=1)
        acc_cls = np.nanmean(acc_cls)
        iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
        mean_iu = np.nanmean(iu)
        freq = hist.sum(axis=1) / hist.sum()
        fwavacc = (freq[freq > 0] * iu[freq > 0]).sum()
        cls_iu = dict(zip(range(self.n_classes), iu))

        return (
            {
                "Overall Acc: \t": acc,
                "Mean Acc : \t": acc_cls,
                "FreqW Acc : \t": fwavacc,
                "Mean IoU : \t": mean_iu,
            },
            cls_iu,
        )

    def reset(self):
        self.confusion_matrix = np.zeros((self.n_classes, self.n_classes))

class averageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## Loader

In [0]:
def get_loader(name):
    """get_loader
    :param name:
    """
    return {
        "pascal": pascalVOCLoader,
    }[name]

logger = logging.getLogger("ptsemseg")

def get_composed_augmentations(aug_dict):
    if aug_dict is None:
        logger.info("Using No Augmentations")
        return None

    augmentations = []
    for aug_key, aug_param in aug_dict.items():
        augmentations.append(key2aug[aug_key](aug_param))
        logger.info("Using {} aug with params {}".format(aug_key, aug_param))
    return Compose(augmentations)

# **Setup**

In [0]:
def setupLoaders(path, img_size, batch_size, num_workers):
    data_loader = get_loader("pascal")

    t_loader = data_loader(
            path,
            is_transform=True,
            split="train",
            img_size=(512, 512),
        )
    print("T-Loader:" +  str(len(t_loader)))

    trainloader = data.DataLoader(
            t_loader,
            batch_size=1,
            num_workers=16,
            shuffle=True,
        )
     
    print("Train-Loader:" +  str(len(trainloader)))
     
    v_loader = pascalVOCLoader(
        root=path,
        is_transform=True,
        split="val",
        img_size=(512, 512),
    )
    print("V-Loader:" +  str(len(v_loader)))
     
    valloader = data.DataLoader(
            v_loader,
            batch_size=1,
            num_workers=16,
        )
    print("Val-Loader:" +  str(len(valloader)))
     
    n_classes = t_loader.n_classes
    print("N Classes: " + str(n_classes))

    return trainloader,valloader,n_classes

# **Training**

In [0]:
def trainModel(trainloader, valloader, model, train_iterations, val_interval, print_interval, num_classes, scheduler, optimizer, loss_fn, name):
    val_loss_meter = averageMeter()
    time_meter = averageMeter()
    best_iou = -100.0
    flag = True
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    running_metrics_val = runningScore(num_classes)
    i = 0

    while i <= train_iterations and flag:
        for (images, labels) in trainloader:
            i += 1
            start_ts = time.time()
            scheduler.step()
            model.train()
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)

            loss = loss_fn(input=outputs, target=labels)

            loss.backward()
            optimizer.step()

            time_meter.update(time.time() - start_ts)

            if (i + 1) % print_interval == 0:
                fmt_str = "Iter [{:d}/{:d}]  Loss: {:.4f}  Time/Image: {:.4f}"
                print_str = fmt_str.format(
                    i + 1,
                    train_iterations,
                    loss.item(),
                    time_meter.avg / 1,
                )

                print(print_str)
                logger.info(print_str)
                writer.add_scalar("loss/train_loss", loss.item(), i + 1)
                time_meter.reset()

            if (i + 1) % val_interval == 0 or (i + 1) == train_iterations:
                model.eval()
                with torch.no_grad():
                    for i_val, (images_val, labels_val) in tqdm(enumerate(valloader)):
                        images_val = images_val.to(device)
                        labels_val = labels_val.to(device)

                        outputs = model(images_val)
                        val_loss = loss_fn(input=outputs, target=labels_val)

                        pred = outputs.data.max(1)[1].cpu().numpy()
                        gt = labels_val.data.cpu().numpy()

                        running_metrics_val.update(gt, pred)
                        val_loss_meter.update(val_loss.item())

                writer.add_scalar("loss/val_loss", val_loss_meter.avg, i + 1)
                logger.info("Iter %d Loss: %.4f" % (i + 1, val_loss_meter.avg))

                score, class_iou = running_metrics_val.get_scores()
                for k, v in score.items():
                    print(k, v)
                    logger.info("{}: {}".format(k, v))
                    writer.add_scalar("val_metrics/{}".format(k), v, i + 1)

                for k, v in class_iou.items():
                    logger.info("{}: {}".format(k, v))
                    writer.add_scalar("val_metrics/cls_{}".format(k), v, i + 1)

                val_loss_meter.reset()
                running_metrics_val.reset()

                if score["Mean IoU : \t"] >= best_iou:
                    best_iou = score["Mean IoU : \t"]
                    state = {
                        "epoch": i + 1,
                        "model_state": model.state_dict(),
                        "optimizer_state": optimizer.state_dict(),
                        "scheduler_state": scheduler.state_dict(),
                        "best_iou": best_iou,
                    }
                    save_path = os.path.join(
                        writer.file_writer.get_logdir(),
                        "{}_{}_best_model.pkl".format(name,"fcn8s"),
                    )
                    torch.save(state, save_path)

            if (i + 1) == train_iterations:
                flag = False
                break

# Running

## Adam Learning Rate Of .001 

In [0]:
image_size = (512,512)
batch_size = 1
num_workers = 16
loss = .001
weight_decay = .0005
data_path = '/content/VOCdevkit/VOC2012'

trainloader,valloader,n_classes = setupLoaders(data_path, image_size, batch_size, num_workers)
Adam_3_Model = torch.nn.DataParallel(fcn8s(),device_ids=range(torch.cuda.device_count()))
optimizer = get_optimizer(Adam_3_Model, "adam", loss, weight_decay)
scheduler = get_scheduler(optimizer, None)
loss = get_loss_function("cross_entropy", False)


In [0]:
train_iterations = 30000
val_interval = 5000
print_interval = 250
name = "Adam_3"
logs_base_dir = "runs/" + name
writer = SummaryWriter(log_dir=logs_base_dir)

trainModel(trainloader, valloader, Adam_3_Model, train_iterations, val_interval, print_interval, n_classes, scheduler, optimizer, loss, name)

## Adam Learning Rate Of .0001



In [0]:
image_size = (512,512)
batch_size = 1
num_workers = 16
loss = .0001
weight_decay = .0005
data_path = '/content/VOCdevkit/VOC2012'

trainloader,valloader,n_classes = setupLoaders(data_path, image_size, batch_size, num_workers)
Adam_4_Model = torch.nn.DataParallel(fcn8s(),device_ids=range(torch.cuda.device_count()))
optimizer = get_optimizer(Adam_4_Model, "adam", loss, weight_decay)
scheduler = get_scheduler(optimizer, None)
loss = get_loss_function("cross_entropy", False)


In [0]:
train_iterations = 30000
val_interval = 5000
print_interval = 500
name = "Adam_4"
logs_base_dir = "runs/" + name
writer = SummaryWriter(log_dir=logs_base_dir)

trainModel(trainloader, valloader, Adam_4_Model, train_iterations, val_interval, print_interval, n_classes, scheduler, optimizer, loss, name)

## Adam Learning Rate Of .00001 

In [0]:
image_size = (512,512)
batch_size = 1
num_workers = 16
loss = .00001
weight_decay = .0005
data_path = '/content/VOCdevkit/VOC2012'

trainloader,valloader,n_classes = setupLoaders(data_path, image_size, batch_size, num_workers)
Adam_5_Model = torch.nn.DataParallel(fcn8s(),device_ids=range(torch.cuda.device_count()))
optimizer = get_optimizer(Adam_5_Model, "adam", loss, weight_decay)
scheduler = get_scheduler(optimizer, None)
loss = get_loss_function("cross_entropy", False)


In [0]:
train_iterations = 30000
val_interval = 5000
print_interval = 250
name = "Adam_5"
logs_base_dir = "runs/" + name
writer = SummaryWriter(log_dir=logs_base_dir)

trainModel(trainloader, valloader, Adam_5_Model, train_iterations, val_interval, print_interval, n_classes, scheduler, optimizer, loss, name)

### View Adam Models in TensorBoard

In [0]:
%load_ext tensorboard
%tensorboard --logdir=runs

## SGD Learning Rate of .001

In [0]:
image_size = (512,512)
batch_size = 1
num_workers = 16
loss = .001
weight_decay = .0005
data_path = '/content/VOCdevkit/VOC2012'

trainloader,valloader,n_classes = setupLoaders(data_path, image_size, batch_size, num_workers)
SGD_3_Model = torch.nn.DataParallel(fcn8s(),device_ids=range(torch.cuda.device_count()))
optimizer = get_optimizer(SGD_3_Model, "sgd", loss, weight_decay)
scheduler = get_scheduler(optimizer, None)
loss = get_loss_function("cross_entropy", False)


In [0]:
train_iterations = 30000
val_interval = 5000
print_interval = 250
name = "SGD_3"
logs_base_dir = "runs/" + name
writer = SummaryWriter(log_dir=logs_base_dir)

trainModel(trainloader, valloader, SGD_3_Model, train_iterations, val_interval, print_interval, n_classes, scheduler, optimizer, loss, name)

## SGD Learning Rate of .0001

In [0]:
image_size = (512,512)
batch_size = 1
num_workers = 16
loss = .0001
weight_decay = .0005
data_path = '/content/VOCdevkit/VOC2012'

trainloader,valloader,n_classes = setupLoaders(data_path, image_size, batch_size, num_workers)
SGD_4_Model = torch.nn.DataParallel(fcn8s(),device_ids=range(torch.cuda.device_count()))
optimizer = get_optimizer(SGD_4_Model, "sgd", loss, weight_decay)
scheduler = get_scheduler(optimizer, None)
loss = get_loss_function("cross_entropy", False)


In [0]:
train_iterations = 30000
val_interval = 5000
print_interval = 250
name = "SGD_4"
logs_base_dir = "runs/" + name
writer = SummaryWriter(log_dir=logs_base_dir)

trainModel(trainloader, valloader, SGD_4_Model, train_iterations, val_interval, print_interval, n_classes, scheduler, optimizer, loss, name)

## SGD Learning Rate of .00001



In [0]:
image_size = (512,512)
batch_size = 1
num_workers = 16
loss = .00001
weight_decay = .0005
data_path = '/content/VOCdevkit/VOC2012'

trainloader,valloader,n_classes = setupLoaders(data_path, image_size, batch_size, num_workers)
SGD_5_Model = torch.nn.DataParallel(fcn8s(),device_ids=range(torch.cuda.device_count()))
optimizer = get_optimizer(SGD_5_Model, "sgd", loss, weight_decay)
scheduler = get_scheduler(optimizer, None)
loss = get_loss_function("cross_entropy", False)


In [0]:
train_iterations = 30000
val_interval = 5000
print_interval = 250
name = "SGD_5"
logs_base_dir = "runs/" + name
writer = SummaryWriter(log_dir=logs_base_dir)

trainModel(trainloader, valloader, SGD_5_Model, train_iterations, val_interval, print_interval, n_classes, scheduler, optimizer, loss, name)

### View SGD Models in TensorBoard

In [0]:
%load_ext tensorboard
%tensorboard --logdir=runs

## Adagrad Learning Rate of .001

In [0]:
image_size = (512,512)
batch_size = 1
num_workers = 16
loss = .001
weight_decay = .0005
data_path = '/content/VOCdevkit/VOC2012'

trainloader,valloader,n_classes = setupLoaders(data_path, image_size, batch_size, num_workers)
Adagrad_3_Model = torch.nn.DataParallel(fcn8s(),device_ids=range(torch.cuda.device_count()))
optimizer = get_optimizer(Adagrad_3_Model, "adagrad", loss, weight_decay)
scheduler = get_scheduler(optimizer, None)
loss = get_loss_function("cross_entropy", False)


In [0]:
train_iterations = 30000
val_interval = 5000
print_interval = 250
name = "Adagrad_3"
logs_base_dir = "runs/" + name
writer = SummaryWriter(log_dir=logs_base_dir)

trainModel(trainloader, valloader, Adagrad_3_Model, train_iterations, val_interval, print_interval, n_classes, scheduler, optimizer, loss, name)

## Adagrad Learning Rate of .0001

In [0]:
image_size = (512,512)
batch_size = 1
num_workers = 16
loss = .0001
weight_decay = .0005
data_path = '/content/VOCdevkit/VOC2012'

trainloader,valloader,n_classes = setupLoaders(data_path, image_size, batch_size, num_workers)
Adagrad_4_Model = torch.nn.DataParallel(fcn8s(),device_ids=range(torch.cuda.device_count()))
optimizer = get_optimizer(Adagrad_4_Model, "adagrad", loss, weight_decay)
scheduler = get_scheduler(optimizer, None)
loss = get_loss_function("cross_entropy", False)


In [0]:
train_iterations = 30000
val_interval = 5000
print_interval = 500
name = "Adagrad_4"
logs_base_dir = "runs/" + name
writer = SummaryWriter(log_dir=logs_base_dir)

trainModel(trainloader, valloader, Adagrad_4_Model, train_iterations, val_interval, print_interval, n_classes, scheduler, optimizer, loss, name)



```
# This is formatted as code
```

## Adagrad Learning Rate of .00001

In [0]:
image_size = (512,512)
batch_size = 1
num_workers = 16
loss = .00001
weight_decay = .0005
data_path = '/content/VOCdevkit/VOC2012'

trainloader,valloader,n_classes = setupLoaders(data_path, image_size, batch_size, num_workers)
Adagrad_5_Model = torch.nn.DataParallel(fcn8s(),device_ids=range(torch.cuda.device_count()))
optimizer = get_optimizer(Adagrad_5_Model, "adagrad", loss, weight_decay)
scheduler = get_scheduler(optimizer, None)
loss = get_loss_function("cross_entropy", False)


In [0]:
train_iterations = 30000
val_interval = 5000
print_interval = 500
name = "Adagrad_5"
logs_base_dir = "runs/" + name
writer = SummaryWriter(log_dir=logs_base_dir)

trainModel(trainloader, valloader, Adagrad_5_Model, train_iterations, val_interval, print_interval, n_classes, scheduler, optimizer, loss, name)

### View Adagrad Models in TensorBoard

In [0]:
%load_ext tensorboard
%tensorboard --logdir=runs

## **Best Model**

Best model is: ..... because ....

Adam -4 we concluded

### Different Schedulers

#### Adam(-4) No Scheduler

In [0]:
image_size = (512,512)
batch_size = 1
num_workers = 16
loss = .0001
weight_decay = .0005
data_path = '/content/VOCdevkit/VOC2012'

trainloader,valloader,n_classes = setupLoaders(data_path, image_size, batch_size, num_workers)
Adam_4_Model = torch.nn.DataParallel(fcn8s(),device_ids=range(torch.cuda.device_count()))
optimizer = get_optimizer(Adam_4_Model, "adam", loss, weight_decay)
loss = get_loss_function("cross_entropy", False)


In [0]:
train_iterations = 30000
val_interval = 5000
print_interval = 500
name = "Adam_4_No"
logs_base_dir = "runs/" + name
writer = SummaryWriter(log_dir=logs_base_dir)

trainModel(trainloader, valloader, Adam_4_Model, train_iterations, val_interval, print_interval, n_classes, scheduler, optimizer, loss, name)

#### Adam(-4) Polynomial Scheduler

In [0]:
image_size = (512,512)
batch_size = 1
num_workers = 16
loss = .0001
weight_decay = .0005
data_path = '/content/VOCdevkit/VOC2012'

trainloader,valloader,n_classes = setupLoaders(data_path, image_size, batch_size, num_workers)
Adam_4_Model = torch.nn.DataParallel(fcn8s(),device_ids=range(torch.cuda.device_count()))
optimizer = get_optimizer(Adam_4_Model, "adam", loss, weight_decay)
scheduler = PolynomialLR(optimizer, 30000)
loss = get_loss_function("cross_entropy", False)


In [0]:
train_iterations = 30000
val_interval = 5000
print_interval = 500
name = "Adam_4_Poly"
logs_base_dir = "runs/" + name
writer = SummaryWriter(log_dir=logs_base_dir)

trainModel(trainloader, valloader, Adam_4_Model, train_iterations, val_interval, print_interval, n_classes, scheduler, optimizer, loss, name)

#### Adam(-4) MultiStepLR Scheduler

In [0]:
image_size = (512,512)
batch_size = 1
num_workers = 16
loss = .0001
weight_decay = .0005
data_path = '/content/VOCdevkit/VOC2012'

trainloader,valloader,n_classes = setupLoaders(data_path, image_size, batch_size, num_workers)
Adam_4_Model = torch.nn.DataParallel(fcn8s(),device_ids=range(torch.cuda.device_count()))
optimizer = get_optimizer(Adam_4_Model, "adam", loss, weight_decay)
scheduler = ExponentialLR(optimizer, 0.95)
loss = get_loss_function("cross_entropy", False)


In [0]:
train_iterations = 30000
val_interval = 5000
print_interval = 500
name = "Adam_4_Expo"
logs_base_dir = "runs/" + name
writer = SummaryWriter(log_dir=logs_base_dir)

trainModel(trainloader, valloader, Adam_4_Model, train_iterations, val_interval, print_interval, n_classes, scheduler, optimizer, loss, name)

##### View Different Schedulers Models in TensorBoard

In [0]:
%load_ext tensorboard
%tensorboard --logdir=runs

### Different Losses

#### Adam(-4) cross_entropy

In [0]:
image_size = (512,512)
batch_size = 1
num_workers = 16
loss = .0001
weight_decay = .0005
data_path = '/content/VOCdevkit/VOC2012'

trainloader,valloader,n_classes = setupLoaders(data_path, image_size, batch_size, num_workers)
Adam_4_Model = torch.nn.DataParallel(fcn8s(),device_ids=range(torch.cuda.device_count()))
optimizer = get_optimizer(Adam_4_Model, "adam", loss, weight_decay)
scheduler = PolynomialLR(optimizer, 30000)
loss = get_loss_function("cross_entropy", False)


In [0]:
train_iterations = 30000
val_interval = 5000
print_interval = 500
name = "Adam_4_Poly_Cross"
logs_base_dir = "runs/" + name
writer = SummaryWriter(log_dir=logs_base_dir)

trainModel(trainloader, valloader, Adam_4_Model, train_iterations, val_interval, print_interval, n_classes, scheduler, optimizer, loss, name)

#### Adam(-4) bootstrapped_cross_entropy

In [0]:
image_size = (512,512)
batch_size = 1
num_workers = 16
loss = .0001
weight_decay = .0005
data_path = '/content/VOCdevkit/VOC2012'

trainloader,valloader,n_classes = setupLoaders(data_path, image_size, batch_size, num_workers)
Adam_4_Model = torch.nn.DataParallel(fcn8s(),device_ids=range(torch.cuda.device_count()))
optimizer = get_optimizer(Adam_4_Model, "adam", loss, weight_decay)
scheduler = PolynomialLR(optimizer, 30000)
loss = functools.partial(bootstrapped_cross_entropy2d, K=64*512)

In [0]:
train_iterations = 30000
val_interval = 5000
print_interval = 500
name = "Adam_4_Poly_Boot"
logs_base_dir = "runs/" + name
writer = SummaryWriter(log_dir=logs_base_dir)

trainModel(trainloader, valloader, Adam_4_Model, train_iterations, val_interval, print_interval, n_classes, scheduler, optimizer, loss, name)

#### Adam(-4) multi_scale_cross_entropy

In [0]:
image_size = (512,512)
batch_size = 1
num_workers = 16
loss = .0001
weight_decay = .0005
data_path = '/content/VOCdevkit/VOC2012'

trainloader,valloader,n_classes = setupLoaders(data_path, image_size, batch_size, num_workers)
Adam_4_Model = torch.nn.DataParallel(fcn8s(),device_ids=range(torch.cuda.device_count()))
optimizer = get_optimizer(Adam_4_Model, "adam", loss, weight_decay)
scheduler = PolynomialLR(optimizer, 30000)
loss = get_loss_function("multi_scale_cross_entropy", False)


In [0]:
train_iterations = 30000
val_interval = 5000
print_interval = 500
name = "Adam_4_Poly_Multi"
logs_base_dir = "runs/" + name
writer = SummaryWriter(log_dir=logs_base_dir)

trainModel(trainloader, valloader, Adam_4_Model, train_iterations, val_interval, print_interval, n_classes, scheduler, optimizer, loss, name)

##### View Different Losses Models in TensorBoard

In [0]:
%load_ext tensorboard
%tensorboard --logdir=runs

## Losing Weights

### Updated Model

In [0]:
class newfcn8s(nn.Module):
    def __init__(self, n_classes=21, learned_billinear=True):
        super(newfcn8s, self).__init__()
        self.learned_billinear = False
        self.n_classes = n_classes
        self.loss = functools.partial(cross_entropy2d, size_average=False)

        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=100),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2, ceil_mode=True),
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2, ceil_mode=True),
        )

        self.conv_block3 = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2, ceil_mode=True),
        )

        self.conv_block4 = nn.Sequential(
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2, ceil_mode=True),
        )

        # self.conv_block5 = nn.Sequential(
        #     nn.Conv2d(512, 512, 3, padding=1),
        #     nn.ReLU(inplace=True),
        #     nn.Conv2d(512, 512, 3, padding=1),
        #     nn.ReLU(inplace=True),
        #     nn.Conv2d(512, 512, 3, padding=1),
        #     nn.ReLU(inplace=True),
        #     nn.MaxPool2d(2, stride=2, ceil_mode=True),
        # )

        self.classifier = nn.Sequential(
            nn.Conv2d(512, 4096, 7),
            nn.ReLU(inplace=True),
            nn.Dropout2d(),
            nn.Conv2d(4096, 4096, 1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(),
            nn.Conv2d(4096, self.n_classes, 1),
        )

        self.score_pool4 = nn.Conv2d(512, self.n_classes, 1)
        self.score_pool3 = nn.Conv2d(256, self.n_classes, 1)

        if self.learned_billinear:
            self.upscore2 = nn.ConvTranspose2d(
                self.n_classes, self.n_classes, 4, stride=2, bias=False
            )
            self.upscore4 = nn.ConvTranspose2d(
                self.n_classes, self.n_classes, 4, stride=2, bias=False
            )
            self.upscore8 = nn.ConvTranspose2d(
                self.n_classes, self.n_classes, 16, stride=8, bias=False
            )

        for m in self.modules():
            if isinstance(m, nn.ConvTranspose2d):
                m.weight.data.copy_(
                    get_upsampling_weight(m.in_channels, m.out_channels, m.kernel_size[0])
                )

    def forward(self, x):
        conv1 = self.conv_block1(x)
        conv2 = self.conv_block2(conv1)
        conv3 = self.conv_block3(conv2)
        conv4 = self.conv_block4(conv3)
        # conv5 = self.conv_block5(conv4)

        score = self.classifier(conv4)

        if self.learned_billinear:
            upscore2 = self.upscore2(score)
            score_pool4c = self.score_pool4(conv4)[
                :, :, 5 : 5 + upscore2.size()[2], 5 : 5 + upscore2.size()[3]
            ]
            upscore_pool4 = self.upscore4(upscore2 + score_pool4c)

            score_pool3c = self.score_pool3(conv3)[
                :, :, 9 : 9 + upscore_pool4.size()[2], 9 : 9 + upscore_pool4.size()[3]
            ]

            out = self.upscore8(score_pool3c + upscore_pool4)[
                :, :, 31 : 31 + x.size()[2], 31 : 31 + x.size()[3]
            ]
            return out.contiguous()

        else:
            score_pool4 = self.score_pool4(conv4)
            score_pool3 = self.score_pool3(conv3)
            score = F.upsample(score, score_pool4.size()[2:])
            score += score_pool4
            score = F.upsample(score, score_pool3.size()[2:])
            score += score_pool3
            out = F.upsample(score, x.size()[2:])

        return out

    def init_vgg16_params(self, vgg16, copy_fc8=True):
        blocks = [
            self.conv_block1,
            self.conv_block2,
            self.conv_block3,
            self.conv_block4,
            # self.conv_block5,
        ]

        ranges = [[0, 4], [5, 9], [10, 16], [17, 23], [24, 29]]
        features = list(vgg16.features.children())

        for idx, conv_block in enumerate(blocks):
            for l1, l2 in zip(features[ranges[idx][0] : ranges[idx][1]], conv_block):
                if isinstance(l1, nn.Conv2d) and isinstance(l2, nn.Conv2d):
                    assert l1.weight.size() == l2.weight.size()
                    assert l1.bias.size() == l2.bias.size()
                    l2.weight.data = l1.weight.data
                    l2.bias.data = l1.bias.data
        for i1, i2 in zip([0, 3], [0, 3]):
            l1 = vgg16.classifier[i1]
            l2 = self.classifier[i2]
            l2.weight.data = l1.weight.data.view(l2.weight.size())
            l2.bias.data = l1.bias.data.view(l2.bias.size())
        n_class = self.classifier[6].weight.size()[0]
        if copy_fc8:
            l1 = vgg16.classifier[6]
            l2 = self.classifier[6]
            l2.weight.data = l1.weight.data[:n_class, :].view(l2.weight.size())
            l2.bias.data = l1.bias.data[:n_class]


In [0]:
image_size = (512,512)
batch_size = 1
num_workers = 16
loss = .0001
weight_decay = .0005
data_path = '/content/VOCdevkit/VOC2012'

trainloader,valloader,n_classes = setupLoaders(data_path, image_size, batch_size, num_workers)
Adam_4_Model = torch.nn.DataParallel(newfcn8s(),device_ids=range(torch.cuda.device_count()))
optimizer = get_optimizer(Adam_4_Model, "adam", loss, weight_decay)
scheduler = PolynomialLR(optimizer, 30000)
loss = get_loss_function("multi_scale_cross_entropy", False)


In [0]:
train_iterations = 30000
val_interval = 5000
print_interval = 500
name = "Adam_4_Lack_Block"
logs_base_dir = "runs/" + name
writer = SummaryWriter(log_dir=logs_base_dir)

trainModel(trainloader, valloader, Adam_4_Model, train_iterations, val_interval, print_interval, n_classes, scheduler, optimizer, loss, name)

##### View Weight Loss Models in TensorBoard

In [0]:
%load_ext tensorboard
%tensorboard --logdir=runs